In [91]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string

a)remove all duplicates from sheet in excel using DATA>remove duplicates
b)using excel concatenate function joined "AgentCorrected_CatName_Primary" and "AgentCorrected_Integer_Primary" column with comma and new target column name is "primary".


In [92]:
def preprocess_text(message):

    new_stopwords=['grocery']
    stpwrd = nltk.corpus.stopwords.words('english')
    stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]','', message.lower())
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #print("message is : ",message)
    
    return message

In [93]:
#df2_cicd.target.value_counts().to_frame('counts').reset_index()
#df2_cicd['label_count'] = df2_cicd.target.replace(label_dict)
#df2_cicd['Counts'] = df2_cicd.groupby(['target']).target.transform('count')
#df2_cicd=df2_cicd.sort_values(by=['Counts'], ascending=True)
df2_cicd=pd.read_csv('../data/TaxML-CICD - Prod_Data.csv')
df2_cicd.drop_duplicates(inplace=True)
df2_cicd.dropna(how='all',inplace=True)
df2_cicd=df2_cicd[(df2_cicd['Agent Corrected CAT Name']!='#REF!')|(df2_cicd['Agent Corrected Integer']!='#REF!')]
df2_cicd['target']=df2_cicd['Agent Corrected CAT Name']+":"+df2_cicd['Agent Corrected Integer']
df2_cicd['Counts'] = df2_cicd.groupby(['target']).target.transform('count')
df2_cicd['input_str'] = df2_cicd[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2_cicd['cleanText']=df2_cicd['input_str'].map(lambda s:preprocess_text(s)) 
#df2_cicd_dup=df2_cicd[df2_cicd.duplicated(subset=['cleanText','target'],keep='last')]
df2_cicd_without_dup=df2_cicd.drop_duplicates(subset=['cleanText','target'],keep='last')
df2 = df2_cicd_without_dup.reset_index(drop=True)
print(df2.shape)

(26912, 19)


In [95]:
X_cicd=df2[['Item','Description','establishment_type','cleanText']]
Y_cicd=df2['target']
X_train_cicd, X_test_cicd,Y_train_cicd, Y_test_cicd=train_test_split(X_cicd,Y_cicd,test_size = .10,random_state = 42)

In [96]:
df = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data_14_01_22.csv', encoding='utf8',engine='python',usecols=['Item','Description','establishment_type','target'])
df1=df.sample(frac=1, random_state=42)
df1 = df1.fillna('')
df1['input_str'] = df1[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['cleanText']=df1['input_str'].map(lambda s:preprocess_text(s)) 
df1_without_dup=df1.drop_duplicates(subset=['cleanText','target'],keep='last')
df1_without_dup['Counts'] = df1_without_dup.groupby(['target']).target.transform('count')
df1= df1_without_dup.reset_index(drop=True)
print(df1.shape)

(136082, 7)


/opt/anaconda3/envs/py365/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [97]:
df3=df1[df1.Counts<15000]   #must in train set
df4=df1[df1.Counts>15000]   #train test split
#print(df3,df4)

In [98]:
X_prev=df3[['Item','Description','establishment_type','cleanText']]
Y_prev=df3['target']
X=df4[['Item','Description','establishment_type','cleanText']]
Y=df4['target']
X_train, X_test,Y_train, Y_test=train_test_split(X,Y,test_size = .20,random_state = 42)

In [99]:
X_train_final_prev=X_train.append(X_train_cicd)
X_train_final=X_train_final_prev.append(X_prev)
X_test_final=X_test.append(X_test_cicd)
Y_train_final_prev=Y_train.append(Y_train_cicd)
Y_train_final=Y_train_final_prev.append(Y_prev)
Y_test_final=Y_test.append(Y_test_cicd)

In [105]:
result=X_test_final
rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', stop_words='english',max_df=0.85)),
       ('tfidf', TfidfTransformer()),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42))])
rf.fit(X_train_final['cleanText'].values,Y_train_final.values)
Y_pred= rf.predict(X_test_final['cleanText'].values)
result['original_cat']=Y_test_final
result['prediction_cat']=Y_pred
result['prediction_cat_confscore']=rf.predict_proba(X_test_final['cleanText'].values).max()
output={'accuracy':accuracy_score(Y_pred,Y_test_final),'precision_score':precision_score(Y_pred,Y_test_final,average='macro'),'recall_score':recall_score(Y_pred,Y_test_final,average='macro')
,'f1_score':f1_score(Y_pred,Y_test_final,average='macro')}
result['confusion_matrix']=str(output)

/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/skle

In [108]:
import datetime
print("Time :{} result :{}".format(datetime.datetime.now(),result['confusion_matrix'][96757]))

Time :2022-02-01 22:54:29.819929 result :{'accuracy': 0.869745869166103, 'precision_score': 0.3807740220916882, 'recall_score': 0.41358596295568073, 'f1_score': 0.37929912910690683}


Time :2022-02-01 17:38:41.818731 result :{'accuracy': 0.8698424968595999, 'precision_score': 0.3807985619689888, 'recall_score': 0.4136497529536392, 'f1_score': 0.37934657549569234}---75%

Time :2022-02-01 17:34:14.198992 result :{'accuracy': 0.817818223974032, 'precision_score': 0.35422299931263246, 'recall_score': 0.26847607737853674, 'f1_score': 0.28068141695978}---50%

Time :2022-02-01 16:07:21.057705 result :{'accuracy': 0.7071594182533549, 'precision_score': 0.37127229896242614, 'recall_score': 0.44096330410202117, 'f1_score': 0.3902240286718021}

Time :2022-02-01 16:15:20.626612 result :{'accuracy': 0.7453943629007991, 'precision_score': 0.40391550958656103, 'recall_score': 0.4922277426356951, 'f1_score': 0.42443260357435253}

Time :2022-02-01 17:22:40.242083 result :{'accuracy': 0.8085435349207051, 'precision_score': 0.35389325636818675, 'recall_score': 0.25924710931465655, 'f1_score': 0.27412973837904125}---25%

In [107]:
import pickle
import datetime
# save the model to disk
primary_model_path = '/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output'
filename_primary= 'finalized_model_01_02_22.sav'
pickle.dump(rf, open(os.path.join(primary_model_path,filename_primary), 'wb'))
# load the model from disk
primary_loaded_model = pickle.load(open(os.path.join(primary_model_path,filename_primary), 'rb'))
# result = loaded_model.score(X_test, Y_test)
result_regression_score = primary_loaded_model.score(df2['cleanText'], df2['target'])
print(result_regression_score)



0.9400639120095124


In [9]:
#file_name='df_traintestdata_{}'.format(datetime.datetime.now())
train_size = 0.8
train_end = int(len(df1)*train_size)
df_train = df1[:train_end]
df_test = df1[train_end:]
train_size_cicd=0.9
train_end_cicd = int(len(df2)*train_size_cicd)
df2_train = df2[:train_end_cicd]
df2_test = df2[train_end_cicd:]
df2_train = df2_train[['Item','Description','establishment_type','input_str','cleanText','target']]
df2_test = df2_test[['Item','Description','establishment_type','input_str','cleanText','target']]
X_train_save = df_train.append(df2_train)
X_test_save = df_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output/df_traintestdata.csv')

In [109]:

misclassifications= result.loc[result['original_cat']!=result['prediction_cat']]

In [110]:
misclassifications.shape

(1348, 8)

In [111]:
misclassifications

,Item,Description,establishment_type,cleanText,original_cat,prediction_cat,prediction_cat_confscore,confusion_matrix
90881,Hiram Walker Peppermint Schnapps 90 proof -1 L...,USA- 90 proof Premium Hiram Walker liqueur inf...,GROCERY,hiram walker peppermint schnapps proof liter u...,CAT_LIQUOR:535,"CAT_ALCOHOL,TEMP_HEATED:109,1",1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
57450,ANDRE BRUT WINE,\N,GROCERY,andre brut wine,CAT_WINE:534,CAT_SPARKLING_WINE:716,1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
64412,Chummy Soju 375ml(16.8%ABV),\N,GROCERY,chummy soju ml abv,CAT_WINE:534,CAT_LIQUOR:535,1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
91950,Bombay Sapphire Gin 1.75,\N,GROCERY,bombay sapphire gin,CAT_LIQUOR:535,"CAT_ALCOHOL,TEMP_UNHEATED:109,1",1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
18083,Angry Orchard Hard Cider Variety Pack 12 pack...,Flavors may be different than shown in picture...,GROCERY,angry orchard hard cider variety pack pack oz ...,CAT_WINE:534,CAT_BEER:533,1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
...,...,...,...,...,...,...,...,...
25087,"Oyster Bay Sauvignon Blanc, 750mL white wine (...",NaN,GROCERY,oyster bay sauvignon blanc ml white wine abv,CAT_WINE:534,"CAT_ALCOHOL,TEMP_UNHEATED:109,1",1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
6136,"Calypso Strawberry Lemonade, (16oz bottle)",\N,GROCERY,calypso strawberry lemonade oz bottle,"CAT_JUICE,TRAIT_PCT_5TO24,TEMP_COLD:110,57,3","CAT_JUICE,TRAIT_PCT_100,TEMP_HEATED:110,51,1",1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
26485,"White Claw Mango, 6pk-12oz can hard seltzer (5...",NaN,LIQUOR,white claw mango pk oz hard seltzer abv liquor,"CAT_ALCOHOL,TEMP_COLD:109,1",CAT_LIQUOR:535,1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."
2662,"Mighty Spark Turkey Patties, Bacon-Bursting (9...",NaN,GROCERY,mighty spark turkey patty baconbursting oz,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,1","CAT_PREPACKAGED_FOOD,TEMP_HEATED:106,1",1.0,"{'accuracy': 0.869745869166103, 'precision_sco..."


In [ ]:

misclassifications_primary= result.loc[result['original_cat_primary']!=result['prediction_cat_primary']]

In [ ]:
misclassifications_primary.sort_values(by=['Item','Description','establishment_type'], ascending=True).head(270)

In [ ]:
misclassifications_primary.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output/misclassifications_primary.csv')

saving model to local

In [12]:
import datetime
print("Time :{} result :{}".format(datetime.datetime.now(),result['confusion_matrix'][0]))

Time :2022-01-25 11:47:43.094480 result :{'accuracy': 0.7932062966031483, 'precision_score': 0.4509416721387717, 'recall_score': 0.5337515891742645, 'f1_score': 0.4726425407676132}


In [ ]:

Time :2022-01-17 18:36:13.829990 result :{'accuracy': 0.8136429968640357, 'precision_score': 0.4811411574576949, 'recall_score': 0.572963521670215, 'f1_score': 0.5053828429504557}